<h1 style="color:#FF4500;"><center>🧡Heart Failure Prediction</center></h1>
<h2 style="color:#228B22;"><center>Visualization And Algorithms</center></h2>
<img src="https://www.pikpng.com/pngl/b/306-3065627_instagram-heart-png-clipart-background-discord-heart-emoji.png" width=300>



<p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
    <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
        <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
            <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
<center><p style="color:#2F4F4F;">Hello, first of all, I wish you all healthy and happy days.Our topic today is to build a high-level forecast model based on the data we have.I think we'll be successful.First of all, I want to start with visual analysis, let's save the prediction model for the end.</p></center>
                <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
                    
                  


<center><h2 style="color:#2F4F4F;">Content</h2></center>
                   


[<center>Cleaning Data</center>](#1)

                        
[<center>Descriptive Statistics</center>](#2)

                        
[<center>Outliers</center>](#3)

                        
[<center>Correlation Map</center>](#4)

                        
[<center>Algorithms</center>](#5)

                        
[<center>Logistic Regression</center>](#6)

                        
[<center>Extreme Gradient Boosting (XGBoost)</center>](#7)

                        
[<center>Random Forests</center>](#8)

                        
[<center>Results</center>](#9)


<a id="1"></a> <br>
<center><h1 style="color:#DC143C;">Cleaning Data</h1></center>
<center><h4 style="color:#2F4F4F;">First, let's take a look at it and clean up the data set we have.</h4></center>

In [ ]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
data.head()

In [ ]:
data.isnull().any()

<center><h4 style="color:#2F4F4F;">Yes, good, there is no missing observation when you look at it.Of course, there may be cases of contradiction, we need to take a look at this, but before that, let's take a look at the descriptive statistics.</h4></center>

<a id="2"></a> <br>

<center><h1 style="color:#DC143C;">Descriptive Statistics</h1></center>
    
<center><h4 style="color:#2F4F4F;">In this section, we will take a look at the data we have and try to make average estimates.</h4></center>



In [ ]:
data.describe().T

<center><h4 style="color:#2F4F4F;">Looking at it, we can say that the data set we have appeals to middle ages.The minimum age is 40 and the maximum is 95.Other than that, I could not comment more here.Now we can take a look at the outliers.</h4></center>

<a id="3"></a> <br>
<center><h1 style="color:#DC143C;">Outliers</h1></center>
<center><h4 style="color:#2F4F4F;">There is a situation like this, we do not know how the data set is collected. It may be wrong to decide on what the exceptions are against, but as a data scientist we have to look at them.
We will make outliers by considering certain variables, these will be numerical variables. I aim to use an interactive library for visualization.</h4></center>
<p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
    <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
<center><h3>Note: Our library is plotly library, those who want to see how it is done can open the cells I have hidden.</h3></center>

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Box(y=data['ejection_fraction'],name='Ejection Fraction',boxpoints='suspectedoutliers',jitter=0.3,marker_color='rgb(52, 152, 219)',line_color='rgb(52, 152, 219)'))

<center><h4 style="color:#2F4F4F;">There doesn't seem to be much of an outlier for the ejection fraction variable so this variable wouldn't be a problem for us.</h4></center>

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=data['platelets'],name='Platelets',boxpoints='suspectedoutliers',jitter=0.3,marker_color='rgb(20, 143, 119)',line_color='rgb(20, 143, 119)'))

<center><h4 style="color:#2F4F4F;">Yes, there are outlier observations for this variable, as I said before, we don't know why they are inconsistent, maybe this outlier could lead to death, and now I noticed that the numbers of this variable require a very large normalization process.</h4></center>

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=data['creatinine_phosphokinase'],name='Creatinine Phosphokinase',boxpoints='suspectedoutliers',jitter=0.3,marker_color='rgb(40, 55, 71)',line_color='rgb(40, 55, 71)'))

<center><h4 style="color:#2F4F4F;">In the same way, there are contradictory observations for this variable.</h4></center>

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=data['serum_creatinine'],name='Serum Creatinine',boxpoints='suspectedoutliers',jitter=0.3,marker_color='rgb(128, 139, 150)',line_color='rgb(128, 139, 150)'))

<center><h4 style="color:#2F4F4F;">We have contradictory observations within this variable.</h4></center>

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=data['serum_sodium'],name='Serum Sodium',boxpoints='suspectedoutliers',jitter=0.3,marker_color='rgb(236, 112, 99)',line_color='rgb(236, 112, 99)'))

<center><h4 style="color:#2F4F4F;">This variable exists inside but that of this variable doesn't affect that much.From here I understand that we won't do anything to these outliers, but I think we need to use a robust algorithm.</h4><center>

<a id="4"></a> <br>
<center><h1 style="color:#DC143C;">Correlation Map</h1></center>
<center><h4 style="color:#2F4F4F;">I think this method can help you make a little more sense.</h4></center>
<p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
    <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
        

<center><h3>Note: I'll use the seaborn library for this,If you don't know how to do it, you can open the hidden cell and look.</h3></center>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(15,10))
ax = sns.heatmap(data.corr(),linewidths=.5,annot=True);

<center><h4 style="color:#2F4F4F;">Looking at it, there is a very, very little correlation between some variables.There is a link between age, smoking and deaths, which is normal as we can guess.This map did not explain the situation to us much, we cannot make an analysis because they are dying because of this, it is completely machine learning.</h4></center>

<a id="5"></a> <br>

<center><h1 style="color:#DC143C;">Algorithms</h1></center>
<p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>

<center><h4 style="color:#2F4F4F;">We will verify multiple models for the data we have.</h4></center>
<p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>

<center><h4 style="color:#2F4F4F;">The models we will use</h4></center>


[<center>Logistic Regression</center>](#30)

[<center>Extreme Gradient Boosting (XGBoost)</center>](#31)

[<center>Random Forests</center>](#32)


    
    
<center><h3>I will work on these algorithms.</h3></center>

<a id="30"></a> <br>
<a id="6"></a> <br>

<center><h1 style="color:#DC143C;">Logistic Regression</h1></center>
<center><h4 style="color:#2F4F4F;">First I will do a common separation for all algorithms.</h4></center>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
y = data['DEATH_EVENT']
x = data.drop(['DEATH_EVENT'],axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=42)

<center><h4 style="color:#2F4F4F;">I will fit all our models according to this separation</h4></center>

In [ ]:
from sklearn.linear_model import LogisticRegression
loj_model = LogisticRegression(solver='lbfgs').fit(x_train,y_train)
y_pred = loj_model.predict(x_test)
print('Accuracy Score For Logistic Regression Model:',accuracy_score(y_test,y_pred))

<a id="31"></a> <br>
<a id="7"></a> <br>

<center><h1 style="color:#DC143C;">Extreme Gradient Boosting (XGBoost)</h1></center>

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier().fit(x_train,y_train)
y_pred = xgb_model.predict(x_test)
print('Simple Accuracy Score For XGBoost Model:',accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
xgb_params = {
    'n_estimators':[100,500,1000,2000],
    'subsample':[0.6,0.8,1.0],
    'max_depth':[3,4,5,6],
    'learning_rate':[0.1,0.01,0.02,0.05],
    
}
xgb = XGBClassifier()
xgb_cv_model = GridSearchCV(xgb,xgb_params,cv=10,n_jobs=-1,verbose=2)
xgb_cv_model.fit(x_train,y_train)

In [ ]:
print('Best Parameters:' + str(xgb_cv_model.best_params_))

In [ ]:
xgb_tuned = XGBClassifier(learning_rate=xgb_cv_model.best_params_['learning_rate'],
                         subsample=xgb_cv_model.best_params_['subsample'],
                         max_depth=xgb_cv_model.best_params_['max_depth'],
                         n_estimators=xgb_cv_model.best_params_['n_estimators']).fit(x_train,y_train)

In [ ]:
y_pred = xgb_tuned.predict(x_test)
print('Verified Accuracy Score For XGBoost Model:',accuracy_score(y_test,y_pred))

<a id="32"></a> <br>
<a id="8"></a> <br>

<center><h1 style="color:#DC143C;">Random Forests</h1></center>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(x_train,y_train)
y_pred = rf_model.predict(x_test)
print('Simple Accuracy Score For RF Model:',accuracy_score(y_test,y_pred))

In [ ]:
rf_params = {
    'max_depth':[2,3,5,8,10],
    'max_features':[2,5,8],
    'n_estimators':[10,500,1000,2000],
    'min_samples_split':[2,5,10]
}
rf = RandomForestClassifier()
rf_cv_model = GridSearchCV(rf,rf_params,cv=10,n_jobs=-1,verbose=2)
rf_cv_model.fit(x_train,y_train)

In [ ]:
print('Best Parameters:' + str(rf_cv_model.best_params_))

In [ ]:
rf = RandomForestClassifier(max_depth=rf_cv_model.best_params_['max_depth'],
                           max_features=rf_cv_model.best_params_['max_features'],
                           n_estimators=rf_cv_model.best_params_['n_estimators'],
                           min_samples_split=rf_cv_model.best_params_['min_samples_split']).fit(x_train,y_train)

In [ ]:
y_pred = rf.predict(x_test)
print('Verified Accuracy Score For RF Model:',accuracy_score(y_test,y_pred))

In [ ]:
# Representation of important values
import matplotlib.pyplot as plt
Importance = pd.DataFrame({'Importance':rf.feature_importances_*100},
                         index=x_train.columns)
Importance.sort_values(by='Importance',
                      axis=0,
                      ascending=True).plot(kind='barh',color='DarkOrange')
plt.xlabel('Importance for variable');

<a id="9"></a> <br>

<center><h1 style="color:#228B22;">Results</h1></center>
<p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>

<center><h3 style="color:#2F4F4F;">Logistic Regression : %80 Accuracy Score</h3></center>
    <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>

<center><h3 style="color:#2F4F4F;">Random Forests : %72 Accuracy Score</h3></center>
        <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
                                                       

<center><h3 style="color:#2F4F4F;">eXtreme Gradient Boosting ( XGBoost ) : %75 Accuracy Score</h3></center>
            <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>

<center><h3 style="color:#228B22;">Thank you all for viewing, stay healthy!</h3></center>